In [2]:
from collections import defaultdict
from itertools import product
import pandas as pd

In [1]:
def build_ngram(tokens, n):
    ngram_counts = defaultdict(int)
    for i in range(len(tokens) - n + 1):
        ngram = tuple(tokens[i:i + n])
        ngram_counts[ngram] += 1
    return ngram_counts

def h_counts(tokens, n):
    ngram_counts = build_ngram(tokens, n)
    if n > 1:
        hcounts = build_ngram(tokens, n - 1)
    else:
        hcounts = {(): len(tokens)}
    return ngram_counts, hcounts

In [3]:
def smoothingfunc(ngram_counts, hcounts, vocabsize, smoothing="add1", k=0.5):
    probs = {}
    if smoothing == "tokentype":
        uniquefolls = defaultdict(set)
        for ngram in ngram_counts:
            h, w = ngram[:-1], ngram[-1]
            uniquefolls[h].add(w)

    for ngram, count in ngram_counts.items():
        h = ngram[:-1]
        hcount = hcounts.get(h, 0)
        if smoothing == "add1":
            probs[ngram] = (count + 1) / (hcount + vocabsize)
        elif smoothing == "addk":
            probs[ngram] = (count + k) / (hcount + k * vocabsize)
        elif smoothing == "tokentype":
            v_h = len(uniquefolls[h]) if h in uniquefolls else 1
            probs[ngram] = (count + v_h) / (hcount + v_h * vocabsize)
        else:
            probs[ngram] = count / hcount if hcount > 0 else 0.0
    return probs

In [4]:
def good_turing_probs(ngram_counts, vocabsize, n):
    counts = list(ngram_counts.values())
    N = len(ngram_counts)
    N1 = sum(1 for c in counts if c == 1)
    
    # Probability for unseen n-grams
    if n == 1:
        unseen_vocab = vocabsize - N
        unseen_prob = (N1 / N) / unseen_vocab if unseen_vocab > 0 else 0.0
    else:
        unseen_vocab = vocabsize**n - N
        unseen_prob = (N1 / N) / unseen_vocab if unseen_vocab > 0 else 0.0
    
    # Probabilities for seen n-grams (MLE)
    total_count = sum(counts)
    probs = {ng: c / total_count for ng, c in ngram_counts.items()}
    
    return probs, unseen_prob


def sentence_prob_good_turing(sentence_tokens, n, probs, unseen_prob):
    prob = 1.0
    sentence_tokens = ["<s>"]*(n-1) + sentence_tokens + ["</s>"]
    for i in range(n-1, len(sentence_tokens)):
        ngram = tuple(sentence_tokens[i-(n-1):i+1])
        prob *= probs.get(ngram, unseen_prob)
    return prob

In [5]:
def deleted_interpolation(sentence_tokens, ngram_probs_list, lambdas , ngram_sizes=[1,2,3,4]):
    prob = 1.0
    max_n = max(ngram_sizes)
    tokens = ["<s>"]*(max_n-1) + sentence_tokens + ["</s>"]

    for i in range(max_n-1, len(tokens)):
        p = 0.0
        for idx, n in enumerate(ngram_sizes):
            ng = tuple(tokens[i-(n-1):i+1])
            p += lambdas[idx] * ngram_probs_list[idx].get(ng, 1e-8)  # tiny fallback
        prob *= p
    return prob

In [6]:
def top_frequencies_table(ngram_counts):
    counts = list(ngram_counts.values())
    C_freq = defaultdict(int)
    
    for c in counts:
        C_freq[c] += 1
    
    table = []
    for c in sorted(C_freq.keys()):
        Nc = C_freq[c]
        C_star = c  # placeholder for advanced Good-Turing C*
        table.append((c, Nc, C_star))
    
    df = pd.DataFrame(table, columns=["C (MLE)", "Nc", "C*"])
    return df.head(100)

In [8]:
sentences = []
tokens_all = []

with open("tokenized_sentences.txt", "r", encoding="utf-8") as f:
    for line in f:
        line_tokens = line.strip().split()
        if line_tokens:
            sentences.append(line_tokens)
            tokens_all.extend(line_tokens)

vocab = set(tokens_all)
V = len(vocab)
print(f"Loaded {len(tokens_all)} tokens, {len(sentences)} sentences, vocab size = {V}")

Loaded 59936 tokens, 1000 sentences, vocab size = 10231


In [9]:
ngram_probs = {}
unseen_probs = {}

def good_turing_probs(ngram_counts, vocabsize, n):
    counts = list(ngram_counts.values())
    N = len(ngram_counts)
    N1 = sum(1 for c in counts if c == 1)

    # Probability for unseen n-grams
    if n == 1:
        unseen_vocab = max(vocabsize - N, 1)  # avoid zero division
        unseen_prob = (N1 / N) / unseen_vocab
    else:
        unseen_vocab = max(vocabsize**n - N, 1)
        unseen_prob = (N1 / N) / unseen_vocab

    # Probabilities for seen n-grams (MLE)
    total_count = sum(counts)
    probs = {ng: c / total_count for ng, c in ngram_counts.items()}

    return probs, unseen_prob

for n in [1, 2, 3, 4]:
    print(f"\nBuilding {n}-gram model...")
    ngram_counts, hcounts = h_counts(tokens_all, n)

    # Different smoothing methods
    add1_probs = smoothingfunc(ngram_counts, hcounts, V, "add1")
    addk_probs = smoothingfunc(ngram_counts, hcounts, V, "addk", k=0.5)
    tokentype_probs = smoothingfunc(ngram_counts, hcounts, V, "tokentype")

    # Good-Turing
    gt_probs, unseen = good_turing_probs(ngram_counts, V, n)
    ngram_probs[n] = gt_probs
    unseen_probs[n] = unseen

    print(f"{n}-gram model done. Example probabilities (Add-1):")
    for i, (ng, p) in enumerate(add1_probs.items()):
        print(f"{ng}: {p:.6f}")
        if i >= 3:
            break



Building 1-gram model...
1-gram model done. Example probabilities (Add-1):
('लोगों',): 0.001981
('को',): 0.015221
('बिलों',): 0.000029
('संबंधी',): 0.000057

Building 2-gram model...
2-gram model done. Example probabilities (Add-1):
('लोगों', 'को'): 0.004533
('को', 'बिलों'): 0.000177
('बिलों', 'संबंधी'): 0.000195
('संबंधी', 'सुविधा'): 0.000195

Building 3-gram model...
3-gram model done. Example probabilities (Add-1):
('लोगों', 'को', 'बिलों'): 0.000195
('को', 'बिलों', 'संबंधी'): 0.000195
('बिलों', 'संबंधी', 'सुविधा'): 0.000195
('संबंधी', 'सुविधा', 'देना'): 0.000195

Building 4-gram model...
4-gram model done. Example probabilities (Add-1):
('लोगों', 'को', 'बिलों', 'संबंधी'): 0.000195
('को', 'बिलों', 'संबंधी', 'सुविधा'): 0.000195
('बिलों', 'संबंधी', 'सुविधा', 'देना'): 0.000195
('संबंधी', 'सुविधा', 'देना', 'ही'): 0.000195


In [11]:
import random

sentences = []
tokens_all = []

with open("tokenized_sentences.txt", "r", encoding="utf-8") as f:
    for line in f:
        line_tokens = line.strip().split()
        if line_tokens:
            sentences.append(line_tokens)
            tokens_all.extend(line_tokens)

# Shuffle sentences for randomness
random.shuffle(sentences)

# Split 50-50
split_idx = len(sentences) // 2
train_sentences = sentences[:split_idx]
test_sentences = sentences[split_idx:]

# Flatten tokens for training
train_tokens = [tok for sent in train_sentences for tok in sent]
test_tokens = [tok for sent in test_sentences for tok in sent]

V = len(set(train_tokens))
print(f"Train tokens: {len(train_tokens)}, Test tokens: {len(test_tokens)}, Vocab size: {V}")


Train tokens: 28250, Test tokens: 31686, Vocab size: 6335


In [12]:
ngram_probs = {}
unseen_probs = {}

for n in [1, 2, 3, 4]:
    ngram_counts, hcounts = h_counts(train_tokens, n)
    
    # Smoothing
    add1_probs = smoothingfunc(ngram_counts, hcounts, V, "add1")
    addk_probs = smoothingfunc(ngram_counts, hcounts, V, "addk", k=0.5)
    tokentype_probs = smoothingfunc(ngram_counts, hcounts, V, "tokentype")
    
    # Good-Turing
    gt_probs, unseen = good_turing_probs(ngram_counts, V, n)
    ngram_probs[n] = gt_probs
    unseen_probs[n] = unseen


In [13]:
def sentence_prob(sentence_tokens, n, probs, unseen_prob):
    prob = 1.0
    sentence_tokens = ["<s>"]*(n-1) + sentence_tokens + ["</s>"]
    for i in range(n-1, len(sentence_tokens)):
        ngram = tuple(sentence_tokens[i-(n-1):i+1])
        prob *= probs.get(ngram, unseen_prob)
    return prob

# Example: probability for test set sentences using 4-gram Good-Turing
sentence_probs = []
for sent in test_sentences:
    p = sentence_prob(sent, 4, ngram_probs[4], unseen_probs[4])
    sentence_probs.append(p)

# Print first 5
for i, p in enumerate(sentence_probs[:5]):
    print(f"Sentence {i+1}: {p:.6e}")


Sentence 1: 4.486316e-142
Sentence 2: 0.000000e+00
Sentence 3: 3.345402e-107
Sentence 4: 3.744029e-320
Sentence 5: 0.000000e+00


In [14]:
import pandas as pd

def top_frequencies_table(ngram_counts):
    counts = list(ngram_counts.values())
    C_freq = defaultdict(int)
    
    for c in counts:
        C_freq[c] += 1
    
    table = []
    for c in sorted(C_freq.keys()):
        Nc = C_freq[c]
        C_star = c  # For simplicity, can be replaced with GT-adjusted C*
        table.append((c, Nc, C_star))
    
    df = pd.DataFrame(table, columns=["C (MLE)", "Nc", "C*"])
    return df.head(100)

# Example for bigrams
bigram_counts, _ = h_counts(train_tokens, 2)
freq_table = top_frequencies_table(bigram_counts)
print(freq_table)


    C (MLE)     Nc   C*
0         1  18216    1
1         2   1704    2
2         3    517    3
3         4    237    4
4         5    104    5
5         6     70    6
6         7     46    7
7         8     37    8
8         9     15    9
9        10     22   10
10       11     10   11
11       12     13   12
12       13     10   13
13       14     12   14
14       15      7   15
15       16      6   16
16       17      8   17
17       18      1   18
18       19      4   19
19       21      7   21
20       22      1   22
21       23      1   23
22       24      2   24
23       25      2   25
24       26      1   26
25       27      2   27
26       28      1   28
27       29      1   29
28       32      2   32
29       34      1   34
30       35      1   35
31       36      1   36
32       37      1   37
33       54      1   54
34       56      1   56
35       58      1   58
36       63      1   63
37       68      1   68
38      105      1  105
39      181      1  181


In [30]:
def deleted_interpolation(sentence_tokens, ngram_probs_list, lambdas, ngram_sizes=[1,2,3,4]):
    prob = 1.0
    max_n = max(ngram_sizes)
    tokens = ["<s>"]*(max_n-1) + sentence_tokens + ["</s>"]
    for i in range(max_n-1, len(tokens)):
        p = 0.0
        for idx, n in enumerate(ngram_sizes):
            ng = tuple(tokens[i-(n-1):i+1])
            p += lambdas[idx] * ngram_probs_list[idx].get(ng, 1e-8)  # fallback tiny prob
        prob *= p
    return prob

# Optimize lambdas (sum=1) for quadrigrams using train set
best_lambdas = [0.25, 0.25, 0.25, 0.25]  # Can run grid search over lambda combinations
ngram_probs_list = [ngram_probs[n] for n in [1,2,3,4]]

# Example: probability of first test sentence
p = deleted_interpolation(test_sentences[0], ngram_probs_list, best_lambdas)
print(f"Deleted interpolated probability: {p:.6e}")


Deleted interpolated probability: 0.000000e+00


In [ ]:
import json

num_sentences = min(1000, len(sentences))
sentence_probs = []

for i in range(num_sentences):
    sent_tokens = sentences[i]
    
    # Example using quadrigram Good-Turing (n=4)
    prob = sentence_prob_good_turing(sent_tokens, 4, ngram_probs[4], unseen_probs[4])
    
    sentence_probs.append({
        "sentence_index": i,
        "sentence": " ".join(sent_tokens),
        "probability": prob
    })

# Save to JSON file
with open("sentence_probabilities.json", "w", encoding="utf-8") as f:
    json.dump(sentence_probs, f, indent=2)

print(f"Saved probabilities for {num_sentences} sentences to sentence_probabilities.json")


Saved probabilities for 1000 sentences to sentence_probabilities.json


: 